# Hybrid Ensemble Model Evaluation

## SARIMA + Prophet + LSTM Ensemble

**Target: SMAPE < 20%**

This notebook evaluates the hybrid ensemble forecasting model that combines:
1. **SARIMA** - Statistical time series model (trend + seasonality)
2. **Prophet** - Facebook's decomposition model (trend + multiple seasonalities)
3. **LSTM** - Deep learning for complex non-linear patterns

The ensemble uses intelligent weight optimization based on validation performance.

In [ ]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add the ml directory to path
ml_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'app', 'ml'))
sys.path.insert(0, ml_path)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12

print("Environment setup complete!")

In [ ]:
# Import forecasters
from sarima_forecaster import SARIMAForecaster, CategoryForecaster

# Try importing ensemble components
try:
    from prophet_forecaster import ProphetForecaster
    PROPHET_AVAILABLE = True
    print("✓ Prophet available")
except ImportError as e:
    PROPHET_AVAILABLE = False
    print(f"✗ Prophet not available: {e}")
    print("  Install with: pip install prophet")

try:
    from lstm_forecaster import LSTMForecaster
    LSTM_AVAILABLE = True
    print("✓ LSTM/TensorFlow available")
except ImportError as e:
    LSTM_AVAILABLE = False
    print(f"✗ LSTM not available: {e}")
    print("  Install with: pip install tensorflow")

try:
    from ensemble_forecaster import EnsembleForecaster, CategoryEnsembleForecaster
    ENSEMBLE_AVAILABLE = PROPHET_AVAILABLE and LSTM_AVAILABLE
    if ENSEMBLE_AVAILABLE:
        print("✓ Ensemble forecaster available")
except ImportError as e:
    ENSEMBLE_AVAILABLE = False
    print(f"✗ Ensemble not available: {e}")

In [ ]:
# Load the data
DATA_PATH = '../../../ml/data/raw/olist_public_dataset_v2.csv'

df = pd.read_csv(DATA_PATH)
df['order_date'] = pd.to_datetime(df['order_date'])

print(f"Loaded {len(df):,} records")
print(f"Date range: {df['order_date'].min()} to {df['order_date'].max()}")
print(f"\nCategories ({df['category'].nunique()}):")
for cat in df['category'].unique():
    print(f"  - {cat}")

In [ ]:
# Prepare weekly data by category
def prepare_weekly_data(df, category):
    """Aggregate to weekly demand for a category."""
    cat_data = df[df['category'] == category].copy()
    weekly = cat_data.groupby(pd.Grouper(key='order_date', freq='W'))['quantity'].sum()
    weekly = weekly[weekly > 0]  # Remove zero weeks
    return weekly

# Helper function for SMAPE
def calculate_smape(actual, predicted):
    """Calculate Symmetric Mean Absolute Percentage Error."""
    denominator = (np.abs(actual) + np.abs(predicted)) / 2.0
    denominator = np.where(denominator == 0, 1, denominator)
    return np.mean(np.abs(actual - predicted) / denominator) * 100

# Get all categories
categories = df['category'].unique()
print(f"Will evaluate {len(categories)} categories")

## 1. Individual Model Performance

First, let's evaluate each model individually on a holdout test set.

In [ ]:
# Evaluate individual models
TEST_SIZE = 8  # 8 weeks holdout

results = []

for category in categories:
    print(f"\n{'='*60}")
    print(f"Evaluating: {category}")
    print(f"{'='*60}")
    
    # Get weekly data
    weekly_data = prepare_weekly_data(df, category)
    
    if len(weekly_data) < 30:
        print(f"Skipping - insufficient data ({len(weekly_data)} weeks)")
        continue
    
    # Train/test split
    train = weekly_data[:-TEST_SIZE]
    test = weekly_data[-TEST_SIZE:]
    
    cat_results = {'category': category, 'test_size': len(test)}
    
    # 1. SARIMA
    try:
        sarima = SARIMAForecaster(seasonal_period=52, use_log_transform=True)
        sarima.fit(train)
        sarima_pred, _, _ = sarima.predict(steps=len(test))
        sarima_pred.index = test.index
        cat_results['sarima_smape'] = calculate_smape(test.values, sarima_pred.values)
        print(f"  SARIMA SMAPE: {cat_results['sarima_smape']:.2f}%")
    except Exception as e:
        cat_results['sarima_smape'] = None
        print(f"  SARIMA failed: {e}")
    
    # 2. Prophet
    if PROPHET_AVAILABLE:
        try:
            prophet = ProphetForecaster()
            prophet.fit(train)
            prophet_pred, _, _ = prophet.predict(steps=len(test))
            prophet_pred.index = test.index
            cat_results['prophet_smape'] = calculate_smape(test.values, prophet_pred.values)
            print(f"  Prophet SMAPE: {cat_results['prophet_smape']:.2f}%")
        except Exception as e:
            cat_results['prophet_smape'] = None
            print(f"  Prophet failed: {e}")
    
    # 3. LSTM
    if LSTM_AVAILABLE:
        try:
            lstm = LSTMForecaster(lookback=12, lstm_units=64, epochs=50)
            lstm.fit(train)
            lstm_pred, _, _ = lstm.predict(steps=len(test))
            lstm_pred.index = test.index
            cat_results['lstm_smape'] = calculate_smape(test.values, lstm_pred.values)
            print(f"  LSTM SMAPE: {cat_results['lstm_smape']:.2f}%")
        except Exception as e:
            cat_results['lstm_smape'] = None
            print(f"  LSTM failed: {e}")
    
    results.append(cat_results)

# Create results DataFrame
results_df = pd.DataFrame(results)
print("\n" + "="*60)
print("Individual Model Results")
print("="*60)
print(results_df.to_string(index=False))

## 2. Ensemble Model Training & Evaluation

Now let's train the ensemble model and compare performance.

In [ ]:
if ENSEMBLE_AVAILABLE:
    ensemble_results = []
    
    for category in categories:
        print(f"\n{'='*60}")
        print(f"Training Ensemble: {category}")
        print(f"{'='*60}")
        
        weekly_data = prepare_weekly_data(df, category)
        
        if len(weekly_data) < 30:
            print(f"Skipping - insufficient data")
            continue
        
        # Train/test split
        train = weekly_data[:-TEST_SIZE]
        test = weekly_data[-TEST_SIZE:]
        
        try:
            # Train ensemble
            ensemble = EnsembleForecaster(
                optimization_method='validation',
                validation_size=8
            )
            ensemble.fit(train, category=category)
            
            # Get predictions
            ensemble_pred, lower, upper = ensemble.predict(steps=len(test))
            ensemble_pred.index = test.index
            
            # Calculate SMAPE
            ensemble_smape = calculate_smape(test.values, ensemble_pred.values)
            
            # Get weights
            weights = ensemble.get_model_weights()
            
            ensemble_results.append({
                'category': category,
                'ensemble_smape': ensemble_smape,
                'sarima_weight': weights['sarima'],
                'prophet_weight': weights['prophet'],
                'lstm_weight': weights['lstm']
            })
            
            print(f"  Ensemble SMAPE: {ensemble_smape:.2f}%")
            print(f"  Weights - SARIMA: {weights['sarima']:.3f}, Prophet: {weights['prophet']:.3f}, LSTM: {weights['lstm']:.3f}")
            
        except Exception as e:
            print(f"  Ensemble failed: {e}")
            ensemble_results.append({
                'category': category,
                'ensemble_smape': None,
                'error': str(e)
            })
    
    ensemble_df = pd.DataFrame(ensemble_results)
    print("\n" + "="*60)
    print("Ensemble Model Results")
    print("="*60)
    print(ensemble_df.to_string(index=False))
else:
    print("Ensemble not available - install prophet and tensorflow")

## 3. Comparison: SARIMA vs Ensemble

In [ ]:
if ENSEMBLE_AVAILABLE and len(ensemble_results) > 0:
    # Merge results
    comparison = pd.merge(
        results_df[['category', 'sarima_smape']],
        ensemble_df[['category', 'ensemble_smape', 'sarima_weight', 'prophet_weight', 'lstm_weight']],
        on='category',
        how='inner'
    )
    
    # Calculate improvement
    comparison['improvement'] = comparison['sarima_smape'] - comparison['ensemble_smape']
    comparison['improvement_pct'] = (comparison['improvement'] / comparison['sarima_smape']) * 100
    
    # Target check
    comparison['meets_target'] = comparison['ensemble_smape'] < 20
    
    print("\n" + "="*80)
    print("SARIMA vs Ensemble Comparison")
    print("="*80)
    print(comparison.to_string(index=False))
    
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"Average SARIMA SMAPE:   {comparison['sarima_smape'].mean():.2f}%")
    print(f"Average Ensemble SMAPE: {comparison['ensemble_smape'].mean():.2f}%")
    print(f"Average Improvement:    {comparison['improvement'].mean():.2f}% ({comparison['improvement_pct'].mean():.1f}% reduction)")
    print(f"\nCategories meeting <20% target: {comparison['meets_target'].sum()}/{len(comparison)}")
    print(f"Categories meeting <25% target: {(comparison['ensemble_smape'] < 25).sum()}/{len(comparison)}")
else:
    print("Cannot compare - ensemble results not available")

## 4. Visualization: Model Comparison

In [ ]:
if ENSEMBLE_AVAILABLE and len(ensemble_results) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. SMAPE Comparison Bar Chart
    ax1 = axes[0, 0]
    x = np.arange(len(comparison))
    width = 0.35
    
    bars1 = ax1.bar(x - width/2, comparison['sarima_smape'], width, label='SARIMA', color='steelblue', alpha=0.8)
    bars2 = ax1.bar(x + width/2, comparison['ensemble_smape'], width, label='Ensemble', color='forestgreen', alpha=0.8)
    
    ax1.axhline(y=20, color='red', linestyle='--', label='Target (20%)')
    ax1.set_xlabel('Category')
    ax1.set_ylabel('SMAPE (%)')
    ax1.set_title('SMAPE Comparison: SARIMA vs Ensemble')
    ax1.set_xticks(x)
    ax1.set_xticklabels([c[:15] for c in comparison['category']], rotation=45, ha='right')
    ax1.legend()
    ax1.set_ylim(0, max(comparison['sarima_smape'].max(), comparison['ensemble_smape'].max()) * 1.2)
    
    # 2. Improvement Chart
    ax2 = axes[0, 1]
    colors = ['forestgreen' if imp > 0 else 'crimson' for imp in comparison['improvement']]
    ax2.barh(comparison['category'], comparison['improvement'], color=colors, alpha=0.8)
    ax2.axvline(x=0, color='black', linewidth=0.5)
    ax2.set_xlabel('SMAPE Improvement (percentage points)')
    ax2.set_title('Ensemble Improvement over SARIMA')
    ax2.set_yticklabels([c[:20] for c in comparison['category']])
    
    # 3. Weight Distribution
    ax3 = axes[1, 0]
    weight_data = comparison[['sarima_weight', 'prophet_weight', 'lstm_weight']].mean()
    colors_pie = ['steelblue', 'coral', 'forestgreen']
    ax3.pie(weight_data, labels=['SARIMA', 'Prophet', 'LSTM'], autopct='%1.1f%%', colors=colors_pie)
    ax3.set_title('Average Model Weights')
    
    # 4. Before/After Summary
    ax4 = axes[1, 1]
    summary_data = {
        'Metric': ['Avg SMAPE', 'Min SMAPE', 'Max SMAPE', 'Std Dev', 'Meeting <20% Target'],
        'SARIMA': [
            f"{comparison['sarima_smape'].mean():.1f}%",
            f"{comparison['sarima_smape'].min():.1f}%",
            f"{comparison['sarima_smape'].max():.1f}%",
            f"{comparison['sarima_smape'].std():.1f}%",
            f"{(comparison['sarima_smape'] < 20).sum()}/{len(comparison)}"
        ],
        'Ensemble': [
            f"{comparison['ensemble_smape'].mean():.1f}%",
            f"{comparison['ensemble_smape'].min():.1f}%",
            f"{comparison['ensemble_smape'].max():.1f}%",
            f"{comparison['ensemble_smape'].std():.1f}%",
            f"{comparison['meets_target'].sum()}/{len(comparison)}"
        ]
    }
    
    ax4.axis('off')
    table = ax4.table(
        cellText=[[summary_data['Metric'][i], summary_data['SARIMA'][i], summary_data['Ensemble'][i]] for i in range(5)],
        colLabels=['Metric', 'SARIMA', 'Ensemble'],
        loc='center',
        cellLoc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.8)
    ax4.set_title('Performance Summary', pad=20)
    
    plt.tight_layout()
    plt.savefig('ensemble_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Saved: ensemble_comparison.png")

## 5. Actual vs Predicted (Best Category)

In [ ]:
if ENSEMBLE_AVAILABLE and len(ensemble_results) > 0:
    # Find best performing category
    best_category = comparison.loc[comparison['ensemble_smape'].idxmin(), 'category']
    best_smape = comparison['ensemble_smape'].min()
    
    print(f"Best performing category: {best_category} (SMAPE: {best_smape:.2f}%)")
    
    # Get data and train ensemble
    weekly_data = prepare_weekly_data(df, best_category)
    train = weekly_data[:-TEST_SIZE]
    test = weekly_data[-TEST_SIZE:]
    
    # Train models
    sarima = SARIMAForecaster(seasonal_period=52, use_log_transform=True)
    sarima.fit(train)
    sarima_pred, sarima_lower, sarima_upper = sarima.predict(steps=len(test))
    sarima_pred.index = test.index
    
    ensemble = EnsembleForecaster(optimization_method='validation', validation_size=8)
    ensemble.fit(train, category=best_category)
    ensemble_pred, ensemble_lower, ensemble_upper = ensemble.predict(steps=len(test))
    ensemble_pred.index = test.index
    ensemble_lower.index = test.index
    ensemble_upper.index = test.index
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # SARIMA
    ax1 = axes[0]
    ax1.plot(train.index[-20:], train.values[-20:], 'b-', label='Training Data', linewidth=2)
    ax1.plot(test.index, test.values, 'g-', label='Actual', linewidth=2, marker='o')
    ax1.plot(sarima_pred.index, sarima_pred.values, 'r--', label='SARIMA Forecast', linewidth=2, marker='s')
    ax1.axvline(x=train.index[-1], color='gray', linestyle=':', alpha=0.5)
    ax1.set_title(f'{best_category} - SARIMA Forecast\nSMAPE: {calculate_smape(test.values, sarima_pred.values):.2f}%')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Weekly Demand')
    ax1.legend()
    ax1.tick_params(axis='x', rotation=45)
    
    # Ensemble
    ax2 = axes[1]
    ax2.plot(train.index[-20:], train.values[-20:], 'b-', label='Training Data', linewidth=2)
    ax2.plot(test.index, test.values, 'g-', label='Actual', linewidth=2, marker='o')
    ax2.plot(ensemble_pred.index, ensemble_pred.values, 'r--', label='Ensemble Forecast', linewidth=2, marker='s')
    ax2.fill_between(ensemble_lower.index, ensemble_lower.values, ensemble_upper.values, color='red', alpha=0.2, label='95% CI')
    ax2.axvline(x=train.index[-1], color='gray', linestyle=':', alpha=0.5)
    ax2.set_title(f'{best_category} - Ensemble Forecast\nSMAPE: {calculate_smape(test.values, ensemble_pred.values):.2f}%')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Weekly Demand')
    ax2.legend()
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('ensemble_actual_vs_predicted.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Saved: ensemble_actual_vs_predicted.png")

## 6. Final Results Summary

In [ ]:
print("\n" + "="*80)
print("FINAL RESULTS: HYBRID ENSEMBLE MODEL EVALUATION")
print("="*80)

if ENSEMBLE_AVAILABLE and len(ensemble_results) > 0:
    print(f"\n📊 MODEL PERFORMANCE")
    print(f"-" * 40)
    print(f"SARIMA Only:")
    print(f"  • Average SMAPE: {comparison['sarima_smape'].mean():.2f}%")
    print(f"  • Categories <20% SMAPE: {(comparison['sarima_smape'] < 20).sum()}/{len(comparison)}")
    print(f"\nEnsemble (SARIMA + Prophet + LSTM):")
    print(f"  • Average SMAPE: {comparison['ensemble_smape'].mean():.2f}%")
    print(f"  • Categories <20% SMAPE: {comparison['meets_target'].sum()}/{len(comparison)}")
    
    print(f"\n📈 IMPROVEMENT")
    print(f"-" * 40)
    print(f"  • Absolute: {comparison['improvement'].mean():.2f} percentage points")
    print(f"  • Relative: {comparison['improvement_pct'].mean():.1f}% reduction in error")
    
    print(f"\n⚖️ AVERAGE MODEL WEIGHTS")
    print(f"-" * 40)
    print(f"  • SARIMA:  {comparison['sarima_weight'].mean()*100:.1f}%")
    print(f"  • Prophet: {comparison['prophet_weight'].mean()*100:.1f}%")
    print(f"  • LSTM:    {comparison['lstm_weight'].mean()*100:.1f}%")
    
    target_met = comparison['meets_target'].sum() == len(comparison)
    print(f"\n🎯 TARGET: SMAPE < 20%")
    print(f"-" * 40)
    if target_met:
        print(f"  ✅ TARGET ACHIEVED! All categories under 20% SMAPE")
    else:
        print(f"  ⚠️ {comparison['meets_target'].sum()}/{len(comparison)} categories meeting target")
        above_target = comparison[~comparison['meets_target']][['category', 'ensemble_smape']]
        print(f"  Categories above target:")
        for _, row in above_target.iterrows():
            print(f"    - {row['category']}: {row['ensemble_smape']:.2f}%")
else:
    print("\n⚠️ Ensemble evaluation could not be completed.")
    print("   Please install required dependencies:")
    print("   pip install prophet tensorflow")

In [ ]:
# Save results to CSV
if ENSEMBLE_AVAILABLE and len(ensemble_results) > 0:
    comparison.to_csv('ensemble_evaluation_results.csv', index=False)
    print("✓ Saved: ensemble_evaluation_results.csv")